# Confounds in fMRI

In [1]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from nistats.hemodynamic_models import spm_hrf
from scipy.ndimage import gaussian_filter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from scipy.stats import pearsonr
from tqdm import tqdm_notebook

%matplotlib inline

In [32]:
class FmriData:
    """ Generates fMRI data.
     
    Parameters
    ----------
    P : int
        Number of conditions
    I : int
        Number of instances per condition
    I_dur : int
        Duration of trial in seconds
    ISIs = list
        List of possible ISIs
    TR : int
        Time to repetition (must be int for simplicity)
    K : tuple
        Voxel dimensions
    ar_rho1 : float
        AR(p) correlation.
    smoothness : int
        Smoothing factor (in sigma)
    """
 
    def __init__(self, P=2, I=20, I_dur=2, ISIs=(4, 5, 6, 7), TR=2, K=(3, 3),
                 ar1_rho=0.5, smoothness=2, noise_factor=1, cond_means=None, cond_stds=None,
                 confound_corr=None, confound_mod='additive'):
        """ Initializes FmriData object. """
        self.P = P
        self.I = I
        self.I_dur = I_dur
        self.ISIs = ISIs
        self.TR = TR
        self.K = K
        self.ar1_rho = ar1_rho
        self.smoothness=smoothness
        self.noise_factor = noise_factor
        self.cond_means = cond_means
        self.cond_stds = cond_stds
        self.confound_corr = confound_corr
        self.confound_mod = 'additive'
        
        self.X = None
        self.y = None
        self.conds = None
        self.conf = None
        self.conf_conv = None
        self.V = None
        self.hrf = None
    
    def generate_data(self, X=None, single_trial=True):
        """ Generates fMRI data (X, y) with or without confound. """

        if X is None:
            X, conds = self._generate_X(single_trial=single_trial)
        
        y, true_betas = self._generate_y(X, conds)
        self.true_betas = true_betas
        return X, y, conds

    def _generate_confound(self, confound_corr, confound_mod, conds):
        """ Generates a confound with specified correlation to trials. """
        
        i = 0
        weight = 0.0001
        conf = np.random.normal(0, 1, conds.size) + (conds - conds.mean()) * weight
        this_corr = pearsonr(conf, conds)[0]
        diff = confound_corr - this_corr 
        while np.abs(diff) > 0.01:
            if diff > 0:
                weight += 0.0001
            else:
                weight -= 0.0001
            conf = np.random.normal(0, 1, conds.size) + conds * weight
            this_corr = pearsonr(conf, conds)[0]
            diff = confound_corr - this_corr 
            i += 1
            
            if i > 100000:
                raise ValueError("Could not create confound with correlation %.3f" % confound_corr)
        
        return conf
                
    def _generate_X(self, single_trial=True):
        """ Generates X (design matrix). """
        
        confound_corr, confound_mod = self.confound_corr, self.confound_mod
        
        # Generate I trials for P conditions
        conds = np.repeat(np.arange(self.P), self.I)
        conds = np.random.permutation(conds)  # shuffle trials
        self.conds = conds

        if confound_corr is not None:
            conf = self._generate_confound(confound_corr, confound_mod, conds)
            self.conf = conf
        else:
            conf = None

        if len(conds) % len(self.ISIs) != 0:
            raise ValueError("Please choose ISIs which can spread across trials evenly.")

        ISIs = np.repeat(self.ISIs, len(conds) / len(self.ISIs))
        ISIs = np.random.permutation(ISIs)
        run_dur = (np.sum(ISIs) + self.I_dur * len(conds))  # run-duration
        
        osf = 10  # oversampling factor for onsets/hrf
        if single_trial:  # nr of regressors = conditions * trials
            X = np.zeros((run_dur * osf, self.P*self.I))
        else:  # nr regressors = nr conditions
            X = np.zeros((run_dur * osf, self.P))

        current_onset = 0  # start creating onsets
        for i, trial in enumerate(conds):

            if single_trial:
                
                if conf is not None and confound_mod == 'modulation':
                    X[current_onset:(current_onset + self.I_dur * osf + conf[i] * osf), i] = 1
                else:
                    X[current_onset:(current_onset + self.I_dur * osf), i] = 1
            else:
                X[current_onset:(current_onset + self.I_dur * osf), trial] = 1

            this_ITI = self.I_dur * osf + ISIs[i] * osf
            current_onset += this_ITI
    
        # Define HRF
        if self.hrf is None:
            hrf = spm_hrf(tr=self.TR, oversampling=self.TR*osf,
                          time_length=32.0, onset=0.0)
            hrf = hrf / np.max(hrf)  # scale HRF, peak = 1
        else:
            hrf = self.hrf

        if conf is not None and confound_mod == 'additive':
            conf_pred = np.zeros(run_dur * osf)
            conf_pred[X.sum(axis=1) != 0] = np.repeat(conf, repeats=osf)
            X = np.c_[X, conf_pred]

        # Convolve regressors with HRF
        X = np.hstack([np.convolve(X[:, i], hrf)[:run_dur*osf, np.newaxis]
                       for i in range(X.shape[1])])
        
        X = X[::self.TR*osf, :]  # downsample
        X = np.c_[np.ones(X.shape[0]), X]  # stack intercept
        if conf is not None and confound_mod == 'additive':
            conf_conv = X[:, -1]
            self.conf_conv = conf_conv
            X = X[:, :-1]
            
        return X, conds

    def _generate_y(self, X, conds, confound_corr=None, confound_mod='additive', single_trial=True):
        """ Generate signals (y). """
        
        confound_corr, confound_mod = self.confound_corr, self.confound_mod
        
        N = X.shape[0]  # N = (downsampled) timepoints
    
        # Create ar1 covariance matrix and generate noise
        if self.V is None:
            self.V = self._generate_V(N)

        noise = np.random.multivariate_normal(np.zeros(N),
                                              self.V,
                                              size=np.prod(K)).T
        # Create condition means/stds
        if single_trial:
            cond_means = np.array([self.cond_means[i] for i in conds])
            cond_stds = np.array([self.cond_stds[i] for i in conds])
            
            if confound_mod == 'additive' and confound_corr is not None:
                cond_means = cond_means.astype(float) + self.conf
        else:
            cond_means = self.cond_means
            cond_stds = self.cond_stds
        
        # Add intercept effects
        cond_means = np.append(0, cond_means)
        cond_stds = np.append(1, cond_stds)

        # Create true paramaters
        if single_trial: 
            covtmp = np.eye(P*I + 1)
            covtmp *= cond_stds
            true_betas = np.random.multivariate_normal(cond_means, covtmp, size=np.prod(K)).T
        else:
            covtmp = np.eye(P + 1)
            covtmp *= cond_stds
            true_betas = np.random.multivariate_normal(cond_means, covtmp, size=np.prod(K)).T

        # Create signal!
        y = X.dot(true_betas) + noise
        if self.smoothness is not None:
            y = gaussian_filter(y.reshape((N,) + K), 1).reshape(N, np.prod(K))
        
        return y, true_betas
        
    def fit_glm(self, X, y, control_for_conf=False, remove_icept=True):
        """ Fits a GLM (using generalized least squares)."""
        
        if control_for_conf:
            conf = self.conf_conv
            X = np.c_[X, conf]
        
        est_betas = np.zeros((X.shape[1], y.shape[1]))
        stderrs = np.zeros_like(est_betas)
        tvals = np.zeros_like(est_betas)
        
        for i in range(y.shape[1]):
            
            gls_results = sm.GLS(y[:, i], X, sigma=self.V).fit()
            est_betas[:, i] = gls_results.params
            stderrs[:, i] = gls_results.bse
            tvals[:, i] = gls_results.tvalues

        if control_for_conf:
            est_betas = est_betas[:-1, :]
            stderrs = stderrs[:-1, :]
            tvals = tvals[:-1, :]

        if remove_icept:
            est_betas = est_betas[1:, :]
            stderrs = stderrs[1:, :]
            tvals = tvals[1:, :]
        
        return est_betas, stderrs, tvals

    def _generate_V(self, N):
        """ Generates a autocovariance matrix based on a AR(1) model. """
        rho = self.ar1_rho
        cov = rho ** scipy.linalg.toeplitz(np.arange(N))
        return cov * self.noise_factor

    def plot(self, X, y, conds, voxel=(0, 0)):
        """ Plots the design and signal of a particular voxel from a particular run. """
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 5), sharex=True)
        #X, y, conds = self.X, self.y, self.conds
        
        ax[0].set_prop_cycle('color', [plt.cm.Dark2(i) for i in conds])
        ax[0].plot(X[:, 1:])
        ax[0].axhline(0, ls='--', c='k', lw=1)
        ax[0].set_xlim(0, X.shape[0])
        ax[0].set_xlabel('Time (in dynamics)', fontsize=15)
        ax[0].set_ylabel('Amplitude (a.u.)', fontsize=15)

        yresh = y.reshape((y.shape[0],) + self.K)
        ax[1].plot(yresh[:, voxel[0], voxel[1]])
        ax[1].axhline(0, ls='--', c='k', lw=1)
        ax[1].set_xlabel('Time (in dynamics)', fontsize=15)
        
        sns.despine()
        fig.tight_layout()
        fig.show()
        
P = 2  # conditions
I = 40  # trials
I_dur = 1  # trial-duration in seconds
ISIs = [14, 16]
TR = 2  # in seconds
K = (10, 10)  # voxels
ar1_rho = 0.5  # autocorr
smoothness = 1  # spatial smoothness in sigma
noise_factor = 0.1  # scaling for V
cond_means = (0, 0)
cond_stds = (0.1, 0.1)

fmri_gen = FmriData(P=P, I=I, I_dur=I_dur, ISIs=ISIs, TR=TR,
                    K=K, ar1_rho=ar1_rho, smoothness=smoothness,
                    noise_factor=noise_factor, cond_means=cond_means,
                    cond_stds=cond_stds)

3.4681698445780382


# Check: how does it do with null-data?

In [ ]:
iters = 10
mean_acc = np.zeros(iters)
mean_acc2 = np.zeros(iters)
mean_acc3 = np.zeros(iters)
mean_acc4 = np.zeros(iters)

pipe = make_pipeline(StandardScaler(), SVC(kernel='linear'))

for i in tqdm_notebook(range(iters)):
    
    X, y, conds = fmri_gen.generate_data(confound_corr=0.8)
    b, v, t = fmri_gen.fit_glm(X, y, control_for_conf=False)
    mean_acc[i] = cross_val_score(pipe, b, conds, cv=10).mean()
    b, v, t = fmri_gen.fit_glm(X, y, control_for_conf=True)
    mean_acc2[i] = cross_val_score(pipe, b, conds, cv=10).mean()
    mean_acc3[i] = cross_val_score(pipe, t, conds, cv=10).mean()
    mean_acc4[i] = cross_val_score(pipe, v, conds, cv=10).mean()

print(mean_acc.mean())
print(mean_acc2.mean())
print(mean_acc3.mean())
print(mean_acc4.mean())

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

# Testing the 'additive' method (Woolgar et al., 2014)
Seems to work!

In [ ]:
N = 100  # timepoints

onsets_A = np.arange(0, N, 10)
X_gen = np.zeros(N)
X_gen[onsets] = 1
conf = np.random.normal(0, 1, onsets.size)
X_gen[onsets] += conf
hrf = spm_hrf(3, oversampling=1)
X_gen = np.convolve(X_gen, hrf)[:X_gen.shape[0]]
true_beta = 1
y_gen = X_gen*true_beta

X_add = np.zeros(N)
X_add[onsets] = 1
X_add = np.convolve(X_add, hrf)[:X_add.shape[0]]

conf_pred = np.zeros(N)
conf_pred[onsets] = conf
conf_conv = np.convolve(conf_pred, hrf)[:conf_pred.shape[0]]
X_add = np.c_[X_add, conf_conv]
y_add = X_add.dot(np.array([1, 1])[:, np.newaxis])

plt.plot(y_gen)
plt.plot(y_add)
plt.show()

In [ ]:
N = 100  # timepoints

onsets_A = np.arange(0, N, 20)
onsets_B = np.arange(10, N, 20)
X_gen = np.zeros((N, 2))
X_gen[onsets_A, 0] = 1
X_gen[onsets_B, 1] = 1

conds = np.tile([0, 1], 5)
conf = np.random.normal(0, 1, 10) + conds
X_gen[onsets_A, 0] += conf[::2]
X_gen[onsets_B, 1] += conf[1::2]

hrf = spm_hrf(3, oversampling=1)
X_gen = np.hstack([np.convolve(X_gen[:, i], hrf)[:X_gen.shape[0], np.newaxis] for i in range(2)])
plt.plot(X_gen)